# Práctica 5 - Visión por Computador

### Autores

- Pablo Segura López
- Kilian Armas Pérez

In [ ]:
import cv2
from deepface import DeepFace

# documentation https://github.com/serengil/deepface/blob/master/deepface/modules/detection.py
# detector_backends deepface options: 'opencv', 'retinaface', 'mtcnn', 'ssd', 'dlib', 'mediapipe', 'yolov8', 'centerface' or 'skip'
detector = 'mtcnn'

# Webcam connection
cap = cv2.VideoCapture(0)
while True:
    # Read frame
    ret, frame = cap.read()
    if not ret:
        break
    try:
        # Face detection
        faces = DeepFace.extract_faces(frame, detector_backend=detector)

        # Draw filter
        for face in faces:
            x, y, w, h = face['facial_area']['x'], face['facial_area']['y'], face['facial_area']['w'], face['facial_area']['h']
            cv2.rectangle(frame, (x, y), (x + w, y + h), (200, 255, 200), 2)  # Dibuja el rectángulo
            if face['facial_area']['left_eye'] is not None:
                cv2.circle(frame, (face['facial_area']['left_eye'][0],face['facial_area']['left_eye'][1]), 3, (0, 255, 0), 2)
                cv2.circle(frame, (face['facial_area']['right_eye'][0],face['facial_area']['right_eye'][1]), 3, (0, 0, 255), 2)

    except Exception as e:
        print("Error:", e)
    cv2.imshow("Video", frame)

    # Salir si se presiona Esc
    key = cv2.waitKey(1) & 0xFF
    if key == 27:
        break

# Liberar la captura y cerrar ventanas
cap.release()
cv2.destroyAllWindows()